In [ ]:
run_simulation <- function(n1, n0, mu1, M = 1000, B = 200) {
  library(birk); library(caret);
  # Set c_opt values
  
  mu0 <- 0
  c_opt_cz <- mu1 / 2
  c_opt_er <- mu1 / 2
  c_opt_youden <- mu1 / 2
  c_opt_iu <- mu1 / 2
  c_opt_ed <- ((n1 * mu1) + (n0 * mu0)) / (n1 + n0)
  
  # Set maxchi manually if n1 != n0
  if (n1 != n0) {
    predefined_maxchi <- c(0.39, 0.46, 0.51, 0.75, 0.87, 0.96, 1.09, 1.23, 1.33, 1.50, 1.63, 1.72)
    idx <- which(c(0.51, 1.05, 1.68, 2.56) == mu1)
    if (length(idx) == 1) {
      c_opt_maxchi <- predefined_maxchi[(idx - 1) * 3 + match(n0, c(100, 150, 200))]
    } else {
      c_opt_maxchi <- mu1 / 2  # fallback
    }
  } else {
    c_opt_maxchi <- mu1 / 2
  }
  
  c_opt <- c(c_opt_cz, c_opt_er, c_opt_youden, c_opt_maxchi, c_opt_iu, c_opt_ed)
  
  cz = function(data){
    tab = as.matrix(table(data$X, data$status))#####the counts of status at each point of test value
    totpos = sum(tab[, 2])#######the total number of  positive's from the status
    totneg = sum(tab[, 1])#######the total number of negative's from the status
    rdf = data.frame(thresholds = unique(sort(data$X)), d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
    rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
    rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
    rdf$TN = totneg - rdf$FP
    rdf$FN = totpos - rdf$TP
    rdf$se = rdf$TP/totpos
    rdf$sp = rdf$TN/totneg
    rdf$cz = rdf$se*rdf$sp
    rdf = data.frame(na.omit(rdf))
    maxcz = rdf[rdf$cz==max(rdf$cz),1]
    cz_cuts = c(maxcz)
    closest_indices_cz = which.closest(cz_cuts,c_opt_cz)
    closest_cz = cz_cuts[closest_indices_cz] 
    return(closest_cz)
  }
  er = function(data){
    tab = as.matrix(table(data$X, data$status))#####the counts of status at each point of test value
    totpos = sum(tab[, 2])#######the total number of  positive's from the status
    totneg = sum(tab[, 1])#######the total number of negative's from the status
    rdf = data.frame(thresholds = unique(sort(data$X)), d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
    rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
    rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
    rdf$TN = totneg - rdf$FP
    rdf$FN = totpos - rdf$TP
    rdf$se = rdf$TP/totpos
    rdf$sp = rdf$TN/totneg
    rdf$fpr = 1-rdf$sp
    rdf$er = sqrt((rdf$fpr^2)+(rdf$se-1)^2)
    rdf = data.frame(na.omit(rdf))
    miner = rdf[rdf$er==min(rdf$er),1]
    er_cuts = c(miner)
    closest_indices_er = which.closest(er_cuts,c_opt_er)
    closest_er = er_cuts[closest_indices_er] 
    return(closest_er)
  }
  youden = function(data){
    tab = as.matrix(table(data$X, data$status))#####the counts of status at each point of test value
    totpos = sum(tab[, 2])#######the total number of  positive's from the status
    totneg = sum(tab[, 1])#######the total number of negative's from the status
    rdf = data.frame(thresholds = unique(sort(data$X)), d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
    rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
    rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
    rdf$TN = totneg - rdf$FP
    rdf$FN = totpos - rdf$TP
    rdf$se = rdf$TP/totpos
    rdf$sp = rdf$TN/totneg
    rdf$youden = rdf$se+rdf$sp-1
    rdf = data.frame(na.omit(rdf))
    maxyouden = rdf[rdf$youden==max(rdf$youden),1]
    youden_cuts = c(maxyouden)
    closest_indices_y = which.closest(youden_cuts,c_opt_youden)
    closest_youden = youden_cuts[closest_indices_y]
    return(closest_youden)
  }
  index_euclidian = function(data){
    ###Index of Euclidian
    n=nrow(data)
    sum=numeric()
    dif=numeric()
    minimum=function(x){
      for(i in 1:n){
        for(j in 1:n){
          dif[j]=(x[i]-x[j])^2
          j=j+1
        }
        sum [i]=sum(dif)
        i=i+1
        
      }
      return(sum)
    }
    data$distance = minimum(data$X)
    ed = data[data$distance==min(data$distance),1]
    ed_cuts = c(ed)
    closest_indices_ed = which.closest(ed_cuts,c_opt_ed)
    closest_ed = ed_cuts[closest_indices_ed]
    return(closest_ed)
  }
  
  chi_max = function(data){
    tab = as.matrix(table(data$X, data$status))#####the counts of status at each point of test value
    totpos = sum(tab[, 2])#######the total number of  positive's from the status
    totneg = sum(tab[, 1])#######the total number of negative's from the status
    rdf = data.frame(thresholds = unique(sort(data$X)), d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
    rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
    rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
    rdf$TN = totneg - rdf$FP
    rdf$FN = totpos - rdf$TP
    rdf$se = rdf$TP/totpos
    rdf$sp = rdf$TN/totneg
    rdf$fpr = 1-rdf$sp
    rdf$chi = ((rdf$se-rdf$fpr)^2)/
      (((totpos*rdf$se+totneg*rdf$fpr)/(totneg+totpos))*
         (1-(totpos*rdf$se+totneg*rdf$fpr)/(totneg+totpos))*(1/totpos+1/totneg))
    rdf = data.frame(na.omit(rdf))
    maxchi = rdf[rdf$chi==max(rdf$chi),1]
    maxchi_cuts= c(maxchi)
    closest_indices_maxchi = which.closest(maxchi_cuts,c_opt_maxchi)
    closest_maxchi = maxchi_cuts[closest_indices_maxchi]
    return(closest_maxchi)
    
  }
  index_union = function(data){
    tab = as.matrix(table(data$X, data$status))#####the counts of status at each point of test value
    totpos = sum(tab[, 2])#######the total number of  positive's from the status
    totneg = sum(tab[, 1])#######the total number of negative's from the status
    rdf = data.frame(thresholds = unique(sort(data$X)), d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
    rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
    rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
    rdf$TN = totneg - rdf$FP
    rdf$FN = totpos - rdf$TP
    rdf$se = rdf$TP/totpos
    rdf$sp = rdf$TN/totneg
    rdf$iu = abs(rdf$se - rdf$sp)
    iu= rdf[rdf$iu==min(rdf$iu),1]
    iu_cuts = c(iu)
    closest_indices_iu = which.closest(iu_cuts,c_opt_iu)
    closest_iu = iu_cuts[closest_indices_iu]
    return(closest_iu)
  }
  opt_boot=matrix(nrow=B,ncol = 6)
  length=numeric()
  sd_boot=numeric()
  coverage = numeric()
  ub=numeric()
  lb=numeric()
  boot = function(data,B,true_param){
    for(i in 1:B){
      data_boot = data[sample(nrow(data),replace = T),]
      closest_cz_boot = cz(data_boot)
      closest_er_boot = er(data_boot)
      closest_youden_boot = youden(data_boot)
      closest_minp_boot = chi_max(data_boot)
      closest_ed_boot = index_euclidian(data_boot)
      closest_iu_boot = index_union(data_boot)
      opt_boot[i,1:6] = matrix(c(closest_cz_boot,closest_er_boot,closest_youden_boot,closest_minp_boot,
                                 closest_iu_boot,closest_ed_boot),ncol=6,byrow=F)
    }
    colnames(opt_boot)=c("cz","er","youden","minp","iu","ed")
    for(j in 1:6){
      ub[j] = quantile(opt_boot[,j],0.975)
      lb[j] = quantile(opt_boot[,j],0.025)
      length[j] = ub[j]-lb[j]
      sd_boot[j] = sd(opt_boot[,j])
      coverage[j] = c_opt[j]>=lb[j] && c_opt[j]<=ub[j]
      boot_results = c(length,sd_boot,coverage)
    }
    return(boot_results)
  }
  results=matrix(nrow=M,ncol = 6)
  bootstrap_results=matrix(nrow=M,ncol = 18)
  acc_results=matrix(nrow=M,ncol = 30)
  ##Relativebias and MSE
  rb=function(true,estimated){
    rb=(mean(estimated)-true)/true
    return(rb)
  }
  mse_f=function(true,estimated){
    mse_f=mean((estimated-true)^2)
    return(mse_f)
  }
  # Simulation
  set.seed(1)
  for (i in 1:M) {
    x_nondiseased = rnorm(n=n0,mean = mu0, sd = 1)
    x_diseased = rnorm(n=n1,mean=mu1, sd=1)
    X=c(x_nondiseased,x_diseased)
    status = c(rep(0,n0),rep(1,n1))
    data = data.frame(cbind(X,status))
    closest_cz = cz(data)
    closest_er = er(data)
    closest_youden = youden(data)
    closest_ed = index_euclidian(data)
    closest_iu = index_union(data)
    closest_chi = chi_max(data)
    data$cz<-ifelse(data$X>=closest_cz,1,0)
    data$er<-ifelse(data$X>=closest_er,1,0)
    data$youden<-ifelse(data$X>=closest_youden,1,0)
    data$chi<-ifelse(data$X>=closest_chi,1,0)
    data$ed<-ifelse(data$X>=closest_ed,1,0)
    data$iu<-ifelse(data$X>=closest_iu,1,0)
    table_cz = table(data$cz,data$status)
    cz_acc = confusionMatrix(table_cz,positive = "1")
    cz_results = c(cz_acc$overall[1],cz_acc$byClass[1:4])
    table_er = table(data$er,data$status)
    er_acc = confusionMatrix(table_er,positive = "1")
    er_results = c(er_acc$overall[1],er_acc$byClass[1:4])
    table_youden = table(data$youden,data$status)
    youden_acc = confusionMatrix(table_youden,positive = "1")
    youden_results = c(youden_acc$overall[1],youden_acc$byClass[1:4])
    table_chi = table(data$chi,data$status)
    chi_acc = confusionMatrix(table_chi,positive = "1")
    chi_results = c(chi_acc$overall[1],chi_acc$byClass[1:4])
    table_ed = table(data$ed,data$status)
    ed_acc = confusionMatrix(table_ed,positive = "1")
    ed_results = c(ed_acc$overall[1],ed_acc$byClass[1:4])
    table_iu = table(data$iu,data$status)
    iu_acc = confusionMatrix(table_iu,positive = "1")
    iu_results = c(iu_acc$overall[1],iu_acc$byClass[1:4])
    bootstrap = boot(data=data,B=B,true_param = c_opt)
    # Bias & MSE
    results[i,1:6]=matrix(c(closest_cz,closest_er,closest_youden,closest_chi,
                            closest_iu,closest_ed),ncol=6,byrow=F)
    bootstrap_results[i,1:18]=matrix(bootstrap,ncol=18,byrow=F)
    acc_results[i,1:30]=matrix(c(cz_results,er_results,youden_results,chi_results,iu_results,ed_results),ncol=30,byrow=F)
  }  
  colnames(results)=c("cz","er","youden","max_chi","iu","ed")
  colnames(bootstrap_results) = c("cz_length","er_length","youden_length","maxp_length","iu_length","ed_lenght",
                                  "cz_sd","er_sd","youden_sd","maxp_sd","iu_sd","ed_sd","cz_cover","er_cover",
                                  "youden_cover","max_cover","iu_cover","ed_cover")
  colnames(acc_results) = c("cz_acc","cz_se","cz_sp","cz_ppv","cz_npv","er_acc","er_se","er_sp",
                            "er_ppv","er_npv","youden_acc","youden_se","youden_sp","youden_ppv",
                            "youden_npv","chi_acc","chi_se","chi_sp","chi_ppv","chi_npv","iu_acc",
                            "iu_se","iu_sp","iu_ppv","iu_npv","ed_acc","ed_se","ed_sp","ed_ppv",
                            "ed_npv")
  
  
  bias=NULL
  for(l in 1:6){
    bias[l] = rb(c_opt[l],results[,l])
    
  }
  
  mse = NULL
  for(k in 1:6){
    mse[k]=mse_f(c_opt[k],results[,k])
  }
  all_perf_results=t(c(bias,mse))
  all_perf_results=data.frame(all_perf_results)
  colnames(all_perf_results)=c("cz_bias","er_bias","youden_bias","chi_bias","iu_bias","ed_bias",
                               "cz_mse","er_mse","youden_mse","chi_mse","iu_mse","ed_mse")
  
  
  return(list(
    cutoffs = results,
    bootstrap = bootstrap_results,
    accuracy = acc_results,
    performance = all_perf_results,
    params = data.frame(n1 = n1, n0 = n0, mu1 = mu1),
    true_cuts=c_opt
  ))
}
s01=run_simulation(n0=50,n1=50,mu1=0.51,M=1000,B=200)
s01_results=data.frame(cbind(s01$params,dist="normal",s01$cutoffs))
s01_perf=data.frame(cbind(s01$params,dist="normal",s01$performance))
s01_acc=data.frame(cbind(s01$params,dist="normal",s01$accuracy))
s01_boot=data.frame(cbind(s01$params,dist="normal",s01$bootstrap))
#########
s02=run_simulation(n0=100,n1=100,mu1=0.51,M=1000,B=200)
s02_results=data.frame(cbind(s02$params,dist="normal",s02$cutoffs))
s02_perf=data.frame(cbind(s02$params,dist="normal",s02$performance))
s02_acc=data.frame(cbind(s02$params,dist="normal",s02$accuracy))
s02_boot=data.frame(cbind(s02$params,dist="normal",s02$bootstrap))
s02=run_simulation(n0=50,n1=50,mu1=0.51,M=1000,B=200)
s02_results=data.frame(cbind(s02$params,dist="normal",s02$cutoffs))
s02_perf=data.frame(cbind(s02$params,dist="normal",s02$performance))
s02_acc=data.frame(cbind(s02$params,dist="normal",s02$accuracy))
s02_boot=data.frame(cbind(s02$params,dist="normal",s02$bootstrap))
##########
s03=run_simulation(n0=200,n1=200,mu1=0.51,M=1000,B=200)
s03_results=data.frame(cbind(s03$params,dist="normal",s03$cutoffs))
s03_perf=data.frame(cbind(s03$params,dist="normal",s03$performance))
s03_acc=data.frame(cbind(s03$params,dist="normal",s03$accuracy))
s03_boot=data.frame(cbind(s03$params,dist="normal",s03$bootstrap))
######
s04=run_simulation(n0=50,n1=50,mu1=1.05,M=1000,B=200)
s04_results=data.frame(cbind(s04$params,dist="normal",s04$cutoffs))
s04_perf=data.frame(cbind(s04$params,dist="normal",s04$performance))
s04_acc=data.frame(cbind(s04$params,dist="normal",s04$accuracy))
s04_boot=data.frame(cbind(s04$params,dist="normal",s04$bootstrap))
#####
s05=run_simulation(n0=100,n1=100,mu1=1.05,M=1000,B=200)
s05_results=data.frame(cbind(s05$params,dist="normal",s05$cutoffs))
s05_perf=data.frame(cbind(s05$params,dist="normal",s05$performance))
s05_acc=data.frame(cbind(s05$params,dist="normal",s05$accuracy))
s05_boot=data.frame(cbind(s05$params,dist="normal",s05$bootstrap))
#########
s06=run_simulation(n0=200,n1=200,mu1=1.05,M=1000,B=200)
s06_results=data.frame(cbind(s06$params,dist="normal",s06$cutoffs))
s06_perf=data.frame(cbind(s06$params,dist="normal",s06$performance))
s06_acc=data.frame(cbind(s06$params,dist="normal",s06$accuracy))
s06_boot=data.frame(cbind(s06$params,dist="normal",s06$bootstrap))
###
s07=run_simulation(n0=50,n1=50,mu1=1.68,M=1000,B=200)
s07_results=data.frame(cbind(s07$params,dist="normal",s07$cutoffs))
s07_perf=data.frame(cbind(s07$params,dist="normal",s07$performance))
s07_acc=data.frame(cbind(s07$params,dist="normal",s07$accuracy))
s07_boot=data.frame(cbind(s07$params,dist="normal",s07$bootstrap))
#####
s08=run_simulation(n0=100,n1=100,mu1=1.68,M=1000,B=200)
s08_results=data.frame(cbind(s08$params,dist="normal",s08$cutoffs))
s08_perf=data.frame(cbind(s08$params,dist="normal",s08$performance))
s08_acc=data.frame(cbind(s08$params,dist="normal",s08$accuracy))
s08_boot=data.frame(cbind(s08$params,dist="normal",s08$bootstrap))
#########
s09=run_simulation(n0=200,n1=200,mu1=1.68,M=1000,B=200)
s09_results=data.frame(cbind(s09$params,dist="normal",s09$cutoffs))
s09_perf=data.frame(cbind(s09$params,dist="normal",s09$performance))
s09_acc=data.frame(cbind(s09$params,dist="normal",s09$accuracy))
s09_boot=data.frame(cbind(s09$params,dist="normal",s09$bootstrap))
######
s10=run_simulation(n0=50,n1=50,mu1=2.56,M=1000,B=200)
s10_results=data.frame(cbind(s10$params,dist="normal",s10$cutoffs))
s10_perf=data.frame(cbind(s10$params,dist="normal",s10$performance))
s10_acc=data.frame(cbind(s10$params,dist="normal",s10$accuracy))
s10_boot=data.frame(cbind(s10$params,dist="normal",s10$bootstrap))
#####
s11=run_simulation(n0=100,n1=100,mu1=2.56,M=1000,B=200)
s11_results=data.frame(cbind(s11$params,dist="normal",s11$cutoffs))
s11_perf=data.frame(cbind(s11$params,dist="normal",s11$performance))
s11_acc=data.frame(cbind(s11$params,dist="normal",s11$accuracy))
s11_boot=data.frame(cbind(s11$params,dist="normal",s11$bootstrap))
#########
s12=run_simulation(n0=200,n1=200,mu1=2.56,M=1000,B=200)
s12_results=data.frame(cbind(s12$params,dist="normal",s12$cutoffs))
s12_perf=data.frame(cbind(s12$params,dist="normal",s12$performance))
s12_acc=data.frame(cbind(s12$params,dist="normal",s12$accuracy))
s12_boot=data.frame(cbind(s12$params,dist="normal",s12$bootstrap))
######
s13=run_simulation(n0=100,n1=50,mu1=0.51,M=1000,B=200)
s13_results=data.frame(cbind(s13$params,dist="normal",s13$cutoffs))
s13_perf=data.frame(cbind(s13$params,dist="normal",s13$performance))
s13_acc=data.frame(cbind(s13$params,dist="normal",s13$accuracy))
s13_boot=data.frame(cbind(s13$params,dist="normal",s13$bootstrap))
#########
s14=run_simulation(n0=150,n1=50,mu1=0.51,M=1000,B=200)
s14_results=data.frame(cbind(s14$params,dist="normal",s14$cutoffs))
s14_perf=data.frame(cbind(s14$params,dist="normal",s14$performance))
s14_acc=data.frame(cbind(s14$params,dist="normal",s14$accuracy))
s14_boot=data.frame(cbind(s14$params,dist="normal",s14$bootstrap))
##########
s15=run_simulation(n0=200,n1=50,mu1=0.51,M=1000,B=200)
s15_results=data.frame(cbind(s15$params,dist="normal",s15$cutoffs))
s15_perf=data.frame(cbind(s15$params,dist="normal",s15$performance))
s15_acc=data.frame(cbind(s15$params,dist="normal",s15$accuracy))
s15_boot=data.frame(cbind(s15$params,dist="normal",s15$bootstrap))
######
s16=run_simulation(n0=100,n1=50,mu1=1.05,M=1000,B=200)
s16_results=data.frame(cbind(s16$params,dist="normal",s16$cutoffs))
s16_perf=data.frame(cbind(s16$params,dist="normal",s16$performance))
s16_acc=data.frame(cbind(s16$params,dist="normal",s16$accuracy))
s16_boot=data.frame(cbind(s16$params,dist="normal",s16$bootstrap))
#####
s17=run_simulation(n0=150,n1=50,mu1=1.05,M=1000,B=200)
s17_results=data.frame(cbind(s17$params,dist="normal",s17$cutoffs))
s17_perf=data.frame(cbind(s17$params,dist="normal",s17$performance))
s17_acc=data.frame(cbind(s17$params,dist="normal",s17$accuracy))
s17_boot=data.frame(cbind(s17$params,dist="normal",s17$bootstrap))
#########
s18=run_simulation(n0=200,n1=50,mu1=1.05,M=1000,B=200)
s18_results=data.frame(cbind(s18$params,dist="normal",s18$cutoffs))
s18_perf=data.frame(cbind(s18$params,dist="normal",s18$performance))
s18_acc=data.frame(cbind(s18$params,dist="normal",s18$accuracy))
s18_boot=data.frame(cbind(s18$params,dist="normal",s18$bootstrap))
###
s19=run_simulation(n0=100,n1=50,mu1=1.68,M=1000,B=200)
s19_results=data.frame(cbind(s19$params,dist="normal",s19$cutoffs))
s19_perf=data.frame(cbind(s19$params,dist="normal",s19$performance))
s19_acc=data.frame(cbind(s19$params,dist="normal",s19$accuracy))
s19_boot=data.frame(cbind(s19$params,dist="normal",s19$bootstrap))
#####
s20=run_simulation(n0=150,n1=50,mu1=1.68,M=1000,B=200)
s20_results=data.frame(cbind(s20$params,dist="normal",s20$cutoffs))
s20_perf=data.frame(cbind(s20$params,dist="normal",s20$performance))
s20_acc=data.frame(cbind(s20$params,dist="normal",s20$accuracy))
s20_boot=data.frame(cbind(s20$params,dist="normal",s20$bootstrap))
#########
s21=run_simulation(n0=200,n1=50,mu1=1.68,M=1000,B=200)
s21_results=data.frame(cbind(s21$params,dist="normal",s21$cutoffs))
s21_perf=data.frame(cbind(s21$params,dist="normal",s21$performance))
s21_acc=data.frame(cbind(s21$params,dist="normal",s21$accuracy))
s21_boot=data.frame(cbind(s21$params,dist="normal",s21$bootstrap))
######
s22=run_simulation(n0=100,n1=50,mu1=2.56,M=1000,B=200)
s22_results=data.frame(cbind(s22$params,dist="normal",s22$cutoffs))
s22_perf=data.frame(cbind(s22$params,dist="normal",s22$performance))
s22_acc=data.frame(cbind(s22$params,dist="normal",s22$accuracy))
s22_boot=data.frame(cbind(s22$params,dist="normal",s22$bootstrap))
#####
s23=run_simulation(n0=150,n1=50,mu1=2.56,M=1000,B=200)
s23_results=data.frame(cbind(s23$params,dist="normal",s23$cutoffs))
s23_perf=data.frame(cbind(s23$params,dist="normal",s23$performance))
s23_acc=data.frame(cbind(s23$params,dist="normal",s23$accuracy))
s23_boot=data.frame(cbind(s23$params,dist="normal",s23$bootstrap))
#########
s24=run_simulation(n0=200,n1=50,mu1=2.56,M=1000,B=200)
s24_results=data.frame(cbind(s24$params,dist="normal",s24$cutoffs))
s24_perf=data.frame(cbind(s24$params,dist="normal",s24$performance))
s24_acc=data.frame(cbind(s24$params,dist="normal",s24$accuracy))
s24_boot=data.frame(cbind(s24$params,dist="normal",s24$bootstrap))
######